In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [ ]:
#import the dataset and selected the useful variables

data=pd.read_csv("../../data/exam_1_with_outcome_nses.csv")
select_exam_1=data[['idno','site1c','gender1','race1c','age1c','cig1c','chol1','hdl1','sbp1c','diabet1','nSES','F3_PC2','y']]
select_exam_1['race_2']=np.where(select_exam_1['race1c']==3,1,0)

#join the dataset with diagnosis day "fuptt"
data_event=pd.read_csv("../../data/outcomes/event.csv")
select_data_event=data_event[['idno','fuptt']]

event_exam_1=pd.merge(select_exam_1,select_data_event, how="left",on=['idno'])
event_exam_1.count()

In [ ]:
#step2: filling in the missing data in train set

event_exam_1['diabet1'] = event_exam_1['diabet1'].replace(9,0) #fill DK as 0
event_exam_1['diabet1'] = event_exam_1['diabet1'].fillna(value=0)

event_exam_1['cig1c'] = event_exam_1['cig1c'].replace(2,1) #group former and current somker together
event_exam_1['cig1c'] = event_exam_1['cig1c'].fillna(value=0) 


#use mean value to fill in the missing value in continuous variables

ave_hdl1 = event_exam_1.mean()['hdl1']
event_exam_1.loc[pd.isna(event_exam_1['hdl1']),'hdl1'] = ave_hdl1

ave_chol1 = event_exam_1.mean()['chol1']
event_exam_1.loc[pd.isna(event_exam_1['chol1']),'chol1'] = ave_chol1

ave_sbp1c = event_exam_1.mean()['sbp1c']
event_exam_1.loc[pd.isna(event_exam_1['sbp1c']),'sbp1c'] = ave_sbp1c

ave_fuptt = event_exam_1.mean()['fuptt']
event_exam_1.loc[pd.isna(event_exam_1['fuptt']),'fuptt'] = ave_fuptt

event_exam_1 = event_exam_1.drop(["race1c"], axis = 1)
event_exam_1.count()

In [ ]:
#add interaction term to the dataset
event_exam_1['gender_race'] = event_exam_1['gender1']*event_exam_1['race_2'] 
event_exam_1.count()

In [ ]:
#standiazed

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler_index = scaler.fit(event_exam_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']],) #scaler index
scaler_index

df_index = event_exam_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']].index
df_cols = event_exam_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']].columns

X_con = scaler_index.transform(event_exam_1[['age1c','chol1','hdl1','sbp1c','F3_PC2']], copy = True) #standiazed
X_con = pd.DataFrame(X_con, index=df_index, columns=df_cols)

X_cate = event_exam_1.loc[:,~event_exam_1.columns.isin(df_cols)]
X_scale = X_cate.join(X_con)

mesa_scale=pd.DataFrame(data=X_scale) #standiazed dataset
mesa_scale

In [ ]:
#output the preprocessed entire dataset
mesa_scale.to_csv("mesa_preprocess_entire.csv",index=False,sep=',')